In [1]:
# The service area can be defined by a lat/long box, and any driver or user outside this area can be ignored.

In [2]:
import threading, time

import time
import json
import sys
#from cloud_tools import Subscriber
import boto3
from hashlib import sha256, shake_256
import hashlib, json, time, enum 

import AWSIoTPythonSDK

import botocore

from AWSIoTPythonSDK import MQTTLib
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from streamlit_folium import folium_static
import folium.plugins
folium.plugins.MarkerCluster()
import os
import sys
import gzip
import json
from pathlib import Path
import csv
import pandas as pd
import s3fs
import pyarrow as pa
from pyarrow.json import read_json
import pyarrow.parquet as pq
import pygeohash
import jsonschema
from jsonschema.exceptions import ValidationError
from logging import Logger


In [3]:
import sqlite3
conn = sqlite3.connect("User_base.db")
c = conn.cursor()
#conn.close()

def make_hashes_len(password, lenOfhash=int):
    	return shake_256(str.encode(password)).hexdigest(lenOfhash)
 
def create_driver_table():
    c.execute('CREATE TABLE IF NOT EXISTS TaxiData(driver_name TEXT, driver_email TEXT UNIQUE, driver_number NUMBER UNIQUE, taxy_Type TEXT, lat_point NUMBER,lon_point NUMBER ,taxy_id TEXT UNIQUE)')
    conn.commit()


def create_user_table():
    c.execute('CREATE TABLE IF NOT EXISTS UserData(newUserName TEXT, newUserEmail TEXT UNIQUE, newUserNumber NUMBER UNIQUE, newUserType TEXT, lat_point NUMBER,lon_point NUMBER ,newUserId TEXT UNIQUE)')
    conn.commit()


def add_driver_data(driver_name,driver_email,driver_number,taxy_Type,lat_point,lon_point ,taxy_id):
    c.execute('INSERT INTO TaxiData(driver_name,driver_email,driver_number,taxy_Type,lat_point,lon_point, taxy_id) VALUES (?,?,?,?,?,?,?)',(driver_name,driver_email,driver_number,taxy_Type,lat_point,lon_point,taxy_id))
    conn.commit()
    
def add_user_data(newUserName,newUserEmail, newUserNumber,newUserType ,lat_point,lon_point ,newUserId):
    c.execute('INSERT INTO UserData(newUserName,newUserEmail, newUserNumber,newUserType ,lat_point,lon_point ,newUserId) VALUES (?,?,?,?,?,?,?)',(newUserName,newUserEmail, newUserNumber,newUserType ,lat_point,lon_point ,newUserId))
    conn.commit()


def view_all_taxi():
    c.execute('SELECT * FROM TaxiData')
    data = c.fetchall()
    return data

def view_all_user():
    c.execute('SELECT * FROM UserData')
    data = c.fetchall()
    return data


def edit_driver_data(driver_email,driver_number,taxy_Type):
	c.execute("UPDATE TaxiData SET driver_email =?,driver_number=?,taxy_Type=? WHERE taxy_id=?",(driver_email,driver_number,taxy_Type))
	conn.commit()
	data = c.fetchall()
	return data

def edit_user_data(newUserEmail,newUserNumber,newUserType):
	c.execute("UPDATE UserData SET newUserEmail =?,newUserNumber=?,newUserType=? WHERE newUserId=?",(newUserEmail,newUserNumber,newUserType))
	conn.commit()
	data = c.fetchall()
	return data


def delete_taxy_data(taxy_id):
    c.execute('DELETE FROM TaxiData WHERE taxy_id="{}"'.format(taxy_id))
    
def delete_user_data(user_id):
    c.execute('DELETE FROM UserData WHERE user_id="{}"'.format(user_id))
    

In [4]:
create_driver_table();create_user_table()

In [5]:
import requests, enum
import urllib.parse
from geopy.geocoders import Nominatim


def make_hashes(password):
	return shake_256(str.encode(password)).hexdigest(77)


def make_hashes_len(password, lenOfhash=int):
	return shake_256(str.encode(password)).hexdigest(lenOfhash)

def check_hashes(password, hashed_text):
    if make_hashes(password) == hashed_text:
        return hashed_text
    
    return False

geolocator = Nominatim(user_agent="taxy_co")

taxyType = enum.Enum('taxyType',
                  ['Utility', 'Deluxe', 'Luxury', 'RideShare'])
address = '1226 Monument Blvd Concord CA'

name = 'Nick'
email = 'around@mail.com'
number = 6456584564
taxy_Type = taxyType.Utility
city ="Concord"
country ="USA"

locGen = geolocator.geocode(city+','+ country)

class newTaxi :
    
      
    def __init__(self, name, email, number, taxy_Type, city, country):
        
        self.name = name
        self.type = taxy_Type
        self.email = email 
        self.number = number
        
        self.newTaxiid = make_hashes_len((name+str(taxy_Type)+(city+ country)), 5)
        
        self.location = geolocator.geocode(city+','+ country)
        self.latitude = self.location.latitude
        self.longitude = self.location.longitude
        self.city = self.location.raw.get('display_name')
        
        
        
        self.Details = {
            
            'newTaxiDName' : self.name,
            'newTaxyDEmail' : self.email,
            'newTaxyDNumber': self.number,
            
            'newTaxyType' : self.type,
            'newTaxyId': self.newTaxiid,
            'newTaxyCity': self.city,

            'newUserPoint': self.location.point
            } 
    

from logging import Logger

class newUser :
    
      
    def __init__(self, name, email, number, user_Type, city, country):
        #userCount = 0
                 
        self.name = name
        self.type = user_Type
        
        self.email = email 
        self.number = number
        
        self.city = city
        self.country = country
        
        self.newUserid = make_hashes_len((name+str(user_Type)+(city+ country)), 5)
        

        self.location = geolocator.geocode(self.city+','+ self.country)
        self.latitude = self.location.latitude
        self.longitude = self.location.longitude
        self.city = self.location.raw.get('display_name')
        
        #self.userCount = userCount

        
        self.Details = {
            
            'newUser Name' : self.name,
            'newUser Email' : self.email,
            'newUser Number': self.number,
            
            'newUser Type' : self.type,
            'newUser Id': self.newUserid,
            'newUser City': self.city,

            'newUser Point': self.location.point,
            #'newUser Count': self.userCount,
            
            
           #  'Customer BC address': self.address
        } 


In [6]:

import requests, enum
import urllib.parse
from geopy.geocoders import Nominatim
from geopy.distance import distance
from shapely.geometry import Point, Polygon
import pandas as pd
import names
import random


taxyType = enum.Enum('taxyType',
                  ['Utility', 'Deluxe', 'Luxury', 'RideShare'])


driver_df = pd.DataFrame({'Driver_name':[],'taxi_locations':[],'phone_num_generator':[],
                          'Driver_emails':[],'taxyType':[],'taxi_cities':[],'Taxiid':[]})

User_df = pd.DataFrame({'User_name':[],'User_locations':[],'User_num_generator':[],
                          'User_emails':[],'UserType':[],'User_cities':[],'Userid':[]})

# Starting point (lat, lon)
#start = (40.730610, -73.935242)

Driver_names = []
Driver_emails = []
taxi_locations = []


for i in range(10):
    Driver_names.append(names.get_full_name())
    
for i in range(10):
    Driver_emails.append(f'{names.get_full_name().replace(" ", "")}@gmail.com')
taxi_location = (37.7387, 121.3555)

# Distance in meters
distance_in_meters = 1000

for m in [0, 10, 50, 100, 150, 200, 225, 250, 275, 300]:
    taxi_locations.append(distance(meters=distance_in_meters).destination(point=taxi_location, bearing=m))

users_names = []
users_emails = []
users_locations = []

for i in range(10):
    users_names.append(names.get_full_name())
    
for i in range(10):
    users_emails.append(f'{names.get_full_name().replace(" ", "")}@gmail.com')
    
    
users_location = (3.7387, 12.3555)

# Distance in meters
distance_in_meters = 1000

for m in [0, 10, 50, 100, 150, 200, 225, 250, 275, 300]:
    users_locations.append(distance(meters=distance_in_meters).destination(point=users_location, bearing=m))





phone_num_generator = []

User_phone_num_generator = []

def random_phone_num_generator():
    first = str(random.randint(100, 999))
    second = str(random.randint(1, 888)).zfill(3)

    last = (str(random.randint(1, 9998)).zfill(4))
    while last in ['1111', '2222', '3333', '4444', '5555', '6666', '7777', '8888']:
        last = (str(random.randint(1, 9998)).zfill(4))

    return '{}-{}-{}'.format(first, second, last)

for i in range(0, 10):
    phone_num_generator.append(random_phone_num_generator())

for i in range(0, 10):
    User_phone_num_generator.append(random_phone_num_generator())

taxi_cities = []

user_cities = []

for i in range(10):
    taxi_cities.append(random.choice(['Concord CA', 'Walnut Creek CA', 'Pleasant Hill CA', 'Martinez CA']))
    
for i in range(10):
    user_cities.append(random.choice(['Concord CA', 'Walnut Creek CA', 'Pleasant Hill CA', 'Martinez CA']))


taxy_Type = []
User_Type = []


for i in range(10):
    taxy_Type.append(random.choice(['Utility', 'Deluxe', 'Luxury', 'RideShare']))


for i in range(10):
    User_Type.append(random.choice(['FirstTime', 'Frequent', 'VipUser', 'NoAccount']))
    
Taxiid = []
Userid = []

for i in range(10):
    Taxiid.append(make_hashes_len((name+str(taxy_Type)+(city+ country)*i), 5))
for i in range(10):
    Userid.append(make_hashes_len((name+str(taxy_Type)+(city+ country)*i), 5))
    
    
        
driver_df.Driver_name = Driver_names

driver_df.Driver_emails = Driver_emails

driver_df.phone_num_generator = phone_num_generator

driver_df.taxyType = taxy_Type

driver_df.taxi_cities = taxi_cities

driver_df.taxi_locations = taxi_locations    

driver_df.Taxiid = Taxiid


for i in driver_df.values:
    #newTaxi(i[0], i[1], i[2], i[3],i[4], i[5], 'USA')
    print(i)
    
    


['Nathan Fortin' Point(37.74770968896363, 121.3555, 0.0) '473-277-0897'
 'GenevieveZavala@gmail.com' 'RideShare' 'Martinez CA' '309218a370']
['Arthur Gutierez' Point(37.74757279518105, 121.35747030421791, 0.0)
 '726-031-3663' 'DonnaCornely@gmail.com' 'Utility' 'Concord CA'
 '67aa6e3462']
['Stephen Patel' Point(37.74449099759493, 121.3641915856997, 0.0)
 '690-403-0307' 'CarolynHarrel@gmail.com' 'Deluxe' 'Concord CA'
 'd80774c343']
['Gary Miller' Point(37.73713495303966, 121.36667258154954, 0.0)
 '125-291-4481' 'LouiseDavies@gmail.com' 'Utility' 'Martinez CA'
 '7aafcf2181']
['Francesca Rodriguez' Point(37.730897232852584, 121.36117199242729, 0.0)
 '984-078-2582' 'JoseBurnett@gmail.com' 'Utility' 'Pleasant Hill CA'
 '0cc0dd8bc6']
['Emmett Schlick' Point(37.73023358532815, 121.35162016330324, 0.0)
 '704-528-2248' 'DaleCunningham@gmail.com' 'RideShare' 'Pleasant Hill CA'
 'ae9cf19c75']
['Oliver Klos' Point(37.73232890655872, 121.34747843693013, 0.0)
 '650-604-6813' 'JoelPonce@gmail.com' 'De

In [23]:
User_df = pd.DataFrame({'User_name':[],'User_locations':[],'User_num_generator':[],
                          'User_emails':[],'UserType':[],'User_cities':[],'Userid':[]})


User_df.User_name = users_names

User_df.User_emails = users_emails

User_df.User_num_generator = User_phone_num_generator

User_df.UserType = User_Type

User_df.User_cities = user_cities

User_df.User_locations = users_locations

User_df.Userid = Userid


for i in User_df.values:
    #newTaxi(i[0], i[1], i[2], i[3],i[4], i[5], 'USA')
    print(i)


['Debra Darbonne' Point(3.747743307715685, 12.3555, 0.0) '801-335-2718'
 'ThomasCastleberry@gmail.com' 'NoAccount' 'Walnut Creek CA' '309218a370']
['Irma Luther' Point(3.747605918167278, 12.357063228529714, 0.0)
 '740-635-8999' 'PearlCruson@gmail.com' 'NoAccount' 'Pleasant Hill CA'
 '67aa6e3462']
['Pearlie Voigt' Point(3.7445128991659753, 12.36239611801851, 0.0)
 '453-668-9350' 'DanaRoss@gmail.com' 'Frequent' 'Walnut Creek CA'
 'd80774c343']
['Anne Millman' Point(3.7371296009834416, 12.364365404251684, 0.0)
 '673-622-0060' 'VanessaAcosta@gmail.com' 'NoAccount' 'Martinez CA'
 '7aafcf2181']
['Paul Tabb' Point(3.7308682527036647, 12.360001051800037, 0.0)
 '998-304-9772' 'IreneCurry@gmail.com' 'NoAccount' 'Walnut Creek CA'
 '0cc0dd8bc6']
['Ron Marshal' Point(3.7302020633572615, 12.35242110155678, 0.0)
 '931-184-2523' 'MaryHanks@gmail.com' 'NoAccount' 'Pleasant Hill CA'
 'ae9cf19c75']
['Hazel Duncan' Point(3.732305391514845, 12.349134541148905, 0.0)
 '363-742-9194' 'EdwardHarris@gmail.com' 

In [8]:

Driver_name_df = driver_df.get(['Driver_name'])

Driver_emails = driver_df.get(['Driver_emails'])

phone_num_generatordf = driver_df.get(['phone_num_generator'])

taxyTypedf = driver_df.get(['taxyType'])

Taxiiddf = driver_df.get(['Taxiid'])

lats = []
for i in range(10):
    lats.append(driver_df.get(['taxi_locations'][0])[i][0])
lons = []
for i in range(10):
    lons.append(driver_df.get(['taxi_locations'][0])[i][1])

driver_df['lats'] = lats
driver_df['lons'] = lons

Lat_df = driver_df.get(['lats'])

Lon_df = driver_df.get(['lons'])


In [9]:
for i in range(10):
    newUsertest = newTaxi(driver_df.loc[i].get('Driver_name'),driver_df.loc[i].get('Driver_emails'),
                        number = driver_df.loc[i].get('phone_num_generator'), taxy_Type = driver_df.loc[i].get('taxyType'),city=  driver_df.loc[i].get('taxi_cities'), country= 'USA')


    newTaxiDName = newUsertest.Details.get('newTaxiDName')
    newTaxyDEmail = newUsertest.Details.get('newTaxyDEmail')
    latitude = newUsertest.latitude
    longitude = newUsertest.longitude
    newTaxyDNumber = newUsertest.Details.get('newTaxyDNumber')
    newTaxyType = newUsertest.Details.get('newTaxyType')
    newTaxyId = newUsertest.Details.get('newTaxyId')
    newTaxyCity = newUsertest.Details.get('newTaxyCity')
    newTaxyId = newUsertest.Details.get('newTaxyId')

    newTaxiDName = newUsertest.Details.get('newTaxiDName')
    newUserPoint = newUsertest.Details.get('newUserPoint')


    add_driver_data(newTaxiDName, newTaxyDEmail, 
            driver_number = newTaxyDNumber,  taxy_Type = newTaxyType,
            taxy_id = newTaxyId, lat_point=latitude, lon_point=longitude)  


{'newTaxiDName': 'Alice Brown',
 'newTaxyDEmail': 'PenniHutchison@gmail.com',
 'newTaxyDNumber': '549-573-7180',
 'newTaxyType': 'RideShare',
 'newTaxyId': '755d325dee',
 'newTaxyCity': 'Concord, Contra Costa County, CAL Fire Northern Region, California, United States',
 'newUserPoint': Point(37.9768525, -122.0335624, 0.0)}

In [11]:
from geopy.distance import distance
from shapely.geometry import Point, Polygon

# Starting point (lat, lon)
#start = (40.730610, -73.935242)

taxi_location = (37.7387, 121.3555)


# Distance in meters
distance_in_meters = 10000

# list of taxi

taxi_list = []

# Generate a point 1km north of the starting point
taxi1 = distance(meters=distance_in_meters).destination(point=taxi_location, bearing=0)
taxi_list.append(taxi1)

# Generate a point 1km east of the starting point
taxi2 = distance(meters=distance_in_meters).destination(point=taxi_location, bearing=90)
taxi_list.append(taxi2)

# Generate a point 1km south of the starting point
taxi3 = distance(meters=distance_in_meters).destination(point=taxi_location, bearing=180)
taxi_list.append(taxi3)

# Generate a point 1km west of the starting point
taxi4 = distance(meters=distance_in_meters).destination(point=taxi_location, bearing=270)
taxi_list.append(taxi4)



# polygon vertices
#polygon_vertices = Polygon([(40.730610, -73.935242), (40.730620, -73.935242), (40.730620, -73.935252), (40.730610, -73.935252)])
service_area = Polygon([(37.8287, -122.3555), (37.8044, 122.2712), (37.3387, 121.8853), (37.9780, 122.0311), (37.9101, 122.0652), (37.6688, 122.0810)])

service_area.contains(Point(taxi_location))

True

In [12]:
if service_area.contains(Point(taxi_location)):
    
    
    print("Taxi is within the service area.")
else:
    print("Taxi is outside the service area.")


Taxi is within the service area.


In [13]:
import shapely.geometry as geometry

In [14]:
from shapely.geometry import Polygon


def validate_service_area(lat, lon, polygon):
    point = geometry.Point(lon, lat)
    return polygon.contains(point)

# Example usage
coords = [(40.730610, -73.935242), (40.730610, -73.935242), (40.730610, -73.935242), (40.730610, -73.935242)]
polygon = Polygon(coords)
location = (40.730610, -73.935242)
is_valid = validate_service_area(location[0], location[1], polygon)
print(is_valid) # True


False


In [15]:

from shapely import Point




# Define the service area using a lat/long bounding box
service_area = Polygon([(37.8287, -122.3555), (37.8044, 122.2712), (37.3387, 121.8853), (37.9780, 122.0311), (37.9101, 122.0652), (37.6688, 122.0810)])


# Check if a given location (taxi's location) is within the service area
taxi_location = (37.7387, 121.3555)


if service_area.contains(Point(taxi_location)):
    
    
    print("Taxi is within the service area.")
else:
    print("Taxi is outside the service area.")


Taxi is within the service area.


In [16]:
from geopy.distance import distance
from shapely.geometry import Point, Polygon

# Starting point (lat, lon)
#start = (40.730610, -73.935242)
import names
Driver_names = []
taxi_locations = []

taxi_location = (37.7387, 121.3555)

# Distance in meters
distance_in_meters = 10000

for i in range(10):
    Driver_names.append(names.get_full_name())



for m in [0, 90, 180, 270]:
    taxi_locations.append(distance(meters=distance_in_meters).destination(point=taxi_location, bearing=m))
    


In [17]:
taxi_locations

[Point(37.82879626815863, 121.3555, 0.0),
 Point(37.73864540367483, 121.4689516988211, 0.0),
 Point(37.64860235118165, 121.3555, 0.0),
 Point(37.73864540367483, 121.24204830117891, 0.0)]

In [18]:

# list of taxi

taxi_list = []

# Generate a point 1km north of the starting point
taxi1 = distance(meters=distance_in_meters).destination(point=taxi_location, bearing=0)
taxi_list.append(taxi1)

# Generate a point 1km east of the starting point
taxi2 = distance(meters=distance_in_meters).destination(point=taxi_location, bearing=90)
taxi_list.append(taxi2)

# Generate a point 1km south of the starting point
taxi3 = distance(meters=distance_in_meters).destination(point=taxi_location, bearing=180)
taxi_list.append(taxi3)

# Generate a point 1km west of the starting point
taxi4 = distance(meters=distance_in_meters).destination(point=taxi_location, bearing=270)
taxi_list.append(taxi4)



# polygon vertices
#polygon_vertices = Polygon([(40.730610, -73.935242), (40.730620, -73.935242), (40.730620, -73.935252), (40.730610, -73.935252)])
service_area = Polygon([(37.8287, -122.3555), (37.8044, 122.2712), (37.3387, 121.8853), (37.9780, 122.0311), (37.9101, 122.0652), (37.6688, 122.0810)])

service_area.contains(Point(taxi_location))

True

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import folium
from sklearn.preprocessing import OrdinalEncoder
from folium.plugins import MarkerCluster
from streamlit_folium import folium_static
import streamlit as st
import folium.plugins
folium.plugins.MarkerCluster()


def CabTypecolors(Cabdata):
    if Cabdata['Taxy Type'] == 'RideShare':
        return 'green'
    elif Cabdata['Taxy Type'] == 'Utility':
        return 'orange'
    elif Cabdata['Taxy Type'] == 'Deluxe':
        return 'red'
    elif Cabdata['Taxy Type'] == 'Luxury':
        return 'aquamarine'
    
def make_map(mapdata):
    locations = mapdata[['latitude', 'longitude']]
    locationlist = locations.values.tolist()
    map = folium.Map(location = [mapdata['latitude'].mean(), mapdata['longitude'].mean()], zoom_start = 3)

    marker_cluster = MarkerCluster().add_to(map)

    for point in range(0, len(locationlist)):
        folium.Marker(
            locationlist[point], popup=mapdata['taxi_id'][point]+' '+mapdata['taxi_type'][point],
            icon=folium.Icon(color=mapdata["color"][point], icon_color='white', icon='cloud', angle=0, prefix='fa')
            ).add_to(map)

    return folium_static(map)

In [20]:
import math
from math import sin, cos, sqrt, atan2, asin
#The above code will generate a set of 8 coordinates within a 0.5km radius of the given latitude and longitude point.
#The bearing variable can be modified to include more

def generate_nearby_coordinates(lat, lon, radius):
    R = 6371 # Earth's radius in kilometers
    bearing = [0, 45, 90, 135, 180, 225, 270, 315] # Possible bearing values
    coordinates = []
    for b in bearing:
        bearing_rad = b * (math.pi / 180)
        lat1 = lat * (math.pi / 180)
        lon1 = lon * (math.pi / 180)
        lat2 = asin(sin(lat1) * cos(distance/R) + cos(lat1) * sin(distance/R) * cos(bearing_rad))
        lon2 = lon1 + atan2(sin(bearing_rad) * sin(distance/R) * cos(lat1), cos(distance/R) - sin(lat1) * sin(lat2))
        coordinates.append((lat2 * (180 / math.pi), lon2 * (180 / math.pi)))
    return coordinates

# Example usage
coordinates = generate_nearby_coordinates(40.730610, -73.935242, 0.5)
print(coordinates)


TypeError: unsupported operand type(s) for /: 'type' and 'int'

In [ ]:
def unique(list1):
     
    # initialize a null list
    unique_list = []
 
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list


def CabTypecolors(Cabdata):
    if Cabdata['Taxy Type'] == 'RideShare':
        return 'green'
    elif Cabdata['Taxy Type'] == 'Utility':
        return 'orange'
    elif Cabdata['Taxy Type'] == 'Deluxe':
        return 'red'
    elif Cabdata['Taxy Type'] == 'Luxury':
        return 'aquamarine'

In [ ]:

    
def make_map(mapdata):
    locations = mapdata[['latitude', 'longitude']]
    locationlist = locations.values.tolist()
    map = folium.Map(location = [mapdata['latitude'].mean(), mapdata['longitude'].mean()], zoom_start = 3)

    marker_cluster = MarkerCluster().add_to(map)

    for point in range(0, len(locationlist)):
        folium.Marker(
            locationlist[point], popup=mapdata['taxi_id'][point]+' '+mapdata['taxi_type'][point],
            icon=folium.Icon(color=mapdata["color"][point], icon_color='white', icon='cloud', angle=0, prefix='fa')
            ).add_to(map)

    return folium_static(map)

In [ ]:
#pip install names

In [ ]:
import names

for i in range(10):
    print(names.get_full_name())